In [2]:

from pyspark import SparkContext

In [3]:
from pyspark.streaming import StreamingContext

In [4]:
sc=SparkContext(appName="2D_analysis")

In [5]:
sc.setLogLevel("ERROR")

In [6]:
ssc=StreamingContext(sc,10)

In [7]:
ssc.checkpoint("checkpoint")

In [8]:
socket=ssc.socketTextStream("172.31.38.183",3307)

In [9]:
lines=socket.window(20)

In [10]:
type(lines)

pyspark.streaming.dstream.DStream

In [11]:
#lines.pprint()

In [12]:
def get_quardent(line):
    try:
        (x,y)=[float(x) for x in line.split()]
    except:
        print "invalid input"
        return ('invalid input' , 1)

    if x > 0  and y > 0:
        quardent = 'First Quardent'
    elif x < 0 and y > 0:
        quardent ='Second Quardent'
    elif x < 0 and y < 0:
        quardent ='Third Quardent'
    elif x > 0 and y <0:
        quardent ='Fourth Quardent'
    elif x ==0 and y != 0:
        quardent ="lies on y-axis"
    elif x != 0 and y == 0:
        quardent ="lies on the x-axis"
    else:
        quardent ="origin"
 
 #   print quardent
    return (quardent,1)



In [13]:
updateFunctions = lambda new_values,running_count:sum(new_values) + (running_count or 0)

In [14]:
running_counts=lines.map(get_quardent).updateStateByKey(updateFunctions)

In [15]:
running_counts.pprint()

In [16]:
ssc.start()

-------------------------------------------
Time: 2017-09-24 02:20:20
-------------------------------------------

-------------------------------------------
Time: 2017-09-24 02:20:30
-------------------------------------------
('First Quardent', 1)

-------------------------------------------
Time: 2017-09-24 02:20:40
-------------------------------------------
('First Quardent', 3)

-------------------------------------------
Time: 2017-09-24 02:20:50
-------------------------------------------
('First Quardent', 4)

-------------------------------------------
Time: 2017-09-24 02:21:00
-------------------------------------------
('First Quardent', 4)

-------------------------------------------
Time: 2017-09-24 02:21:10
-------------------------------------------
('First Quardent', 4)

-------------------------------------------
Time: 2017-09-24 02:21:20
-------------------------------------------
('First Quardent', 4)

-------------------------------------------
Time: 2017-09-24 02

In [ ]:
ssc.awaitTermination()